In [1]:
### functional transcription


# from IPython.display import display, HTML
import sounddevice as sd
import numpy as np
import queue
from scipy.io.wavfile import write
from faster_whisper import WhisperModel
from ipywidgets import widgets, Layout, Button, VBox, Textarea, HBox, HTML
from IPython.display import display, Markdown

# Initialisieren des Whisper-Modells
# model = WhisperModel("tiny", device="cuda")
# model = WhisperModel("tiny", device="cpu")
model = WhisperModel("small", device="cpu")

# Audioaufnahme-Einstellungen
sample_rate = 16000  # Sampling-Rate
audio_queue = queue.Queue()

audio_data = []
my_text = Textarea()

# callbacks & fcns
def audio_callback(indata, frames, time, status):
    """Diese Funktion wird von sounddevice während der Audioaufnahme aufgerufen."""
    audio_queue.put(indata.copy())

stream = sd.InputStream(callback=audio_callback, samplerate=sample_rate, channels=1, dtype='int16')

def start_recording(b=None):
    """Startet die Audioaufnahme."""
    audio_queue.queue.clear()  # Bereinigt die Queue vor der Aufnahme
    stream.start()

def interrupt_recording(b=None):
    """Stoppt die Audioaufnahme und startet die Transkription."""
    global audio_data
    
    stream.stop()

    if len(audio_data) > 20:
        audio_data = audio_data[-20:]
    
    while not audio_queue.empty():
        audio_data.append(audio_queue.get())
    start_recording()
    if audio_data:
        audio_np = np.concatenate(audio_data, axis=0)
        write('output.wav', sample_rate, audio_np.astype(np.int16))  # Schreibt die WAV-Datei
        transcribe_audio('output.wav')

def transcribe_audio(audio_path):
    """Transkribiert das aufgenommene Audio."""
    segments, _ = model.transcribe(audio_path, language="de", word_timestamps=True)
    transcription = " ".join([segment.text for segment in segments])
    global my_text
    my_text.value = transcription
    #print(transcription) # Aktualisiert das Text-Widget mit der Transkription

def reset_transcription(b):
    """Setzt die Transkription zurück."""
    text_output.value = ""

my_text

Textarea(value='')

In [ ]:

from time import sleep

start_recording()
for i in range(100):
    sleep(0.5)
    interrupt_recording()
